<a href="https://colab.research.google.com/github/ASO1801023/AI_teach2020/blob/master/2020AI0305_fishvideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 動画ファイルから画面の変化を検出して、画像フィルに保存する

### ライブラリのimport

In [1]:
# ライブラリのimport
import cv2
# OSの操作用オブジェクト
import os
# urlダウンロード用
import urllib.request as req

### 処理用変数の定義とディレクトリの生成処理

In [2]:
# 変数の定義
# 前回の画像データ用変数
img_last = None
# ファイル番号
no = 0
# ファイル保存ディレクトリ用文字列
save_dir = "./exfish"
# OSにディレクトリ作成を命令する
os.makedirs( save_dir, exist_ok=True ) # 存在する場合は何もしない

### 動画ファイルを終わりまで1フレームずつ読みこみ、比較してファイル保存

In [ ]:
# 動画ファイルをダウンロード
url = "https://github.com/masatokg/sample_photo/raw/master/fish.mp4"
save_file = "fish.mp4"
# ファイルをダウンロードして保存
req.urlretrieve( url, save_file )

# oepnCVでファイルの画像データを読み込む
cap = cv2.VideoCapture( save_file )

# 次のフレームを終わりまで順次無限ループで読み込む
while( True ):
  # フレームを読み込む（終わりだったら第一戻り値にFalseが戻る）
  is_ok, frame = cap.read()
  # もし、フレームが読み込めなかったら（終わりなら）
  if not is_ok:
    # メッセージを出して無限ループをbreakする
    print("end.")
    break
  # if文終わり
  # 切り抜き処理などのためにフレーム画像のサイズを640*340に整える
  frame = cv2.resize( frame, (640, 360) )
  # グレースケール画像に変換
  gray = cv2.cvtColor( frame, cv2.COLOR_BGR2GRAY )
  # 少しガウスぼかしをかけて輪郭を太くする
  gray = cv2.GaussianBlur( gray, (15,15), 0 )
  # 閾値を指定してブラックアンドホワイトの2極画像に変換する
  img_b = cv2.threshold( gray, 127, 255, cv2.THRESH_BINARY )[1]
  
  # もし前回記憶したブラックアンドホワイトの画像が存在したら、差分を検出
  if not img_last is None:
    # 差分となる値を絶対値で取得
    frame_diff = cv2.absdiff( img_last, img_b )

    # 差分の輪郭の特徴（座標）のリストを取得
    cnts = cv2.findContours( frame_diff, 
                            cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    # 差分の数だけループして差分の座標の領域の画像データを切り抜いてファイルに保存
    for pt in cnts:
      # 差分の領域の座標（と縦幅、横幅）を変数に格納
      x, y, w, h = cv2.boundingRect(pt)
      # 領域があまりに小さすぎたり、大きすぎたら、ノイズとして無視する
      if w< 100 or w > 500: continue

      # フレームから指定した座標領域の画像データを切り抜く
      imgex = frame[ y:y+h, x:x+w ]

      # ファイル保存処理
      # ファイル名組み立て
      outfile = save_dir + "/" + str(no) + ".jpg"
      # ファイル保存命令
      cv2.imwrite( outfile, imgex )

      # ファイル番号を+1する
      no += 1
    # for文終わり
  # if文終わり
  # ブラックアンドホワイトの画像を記憶する
  img_last = img_b
# while文終了
# 画像ファイルの読込を開放
cap.release()
# 最終終了メッセージ
print("ok")


### ファイルをzipに圧縮してダウンロード

In [ ]:
# ダウンロードしたいフォルダを zip 圧縮する
! zip -r /content/download.zip /content/exfish

# 圧縮した zip ファイルをダウンロードする
from google.colab import files
files.download("/content/download.zip")
